# Modelagem de Tópicos

São poucas as quantidades de modelos que podemos aplicar na maneira que os nossos dados se encontram neste exato momento. Com isso, para que tenhamos mais variáveis quantitativas que ajudam na análise, o intuito é dividir os tweets da família Bolsonaro em tópicos.

In [1]:
from sklearn.feature_extraction import text 
import pandas as pd
import numpy as np
import re
import string
import unicodedata
from gensim import matutils, models
import scipy.sparse
import nltk

Como vamos usar diversos recursos do nltk, vale a pena baixar todo seu conteúdo (uma aba vai abrir e é só clicar em baixar tudo):

In [2]:
#nltk.download()

## Gerando a Document Term Matrix

Usando as stop words em português obtidas no site https://gist.github.com/alopes/5358189, podemos limpar nossos textos dos tweets de modo a transformá-los em uma Document Term Matrix.

In [3]:
tweeys_data = pd.read_csv("../../data/tweets/preprocessed_tweets.csv", sep="~", index_col=0)
tweeys_data["date"] = pd.to_datetime(tweeys_data["date"])
tweeys_data.set_index("date", inplace=True)
tweeys_data.head()

,full_text,hashtags,user_mentions,media_type,status_reply,name,retweet_count,favorite_count,year,month,day,hour,minute,weekday,has_hashtags,has_mentions,has_media
date,,,,,,,,,,,,,,,,,
2020-07-27 20:51:13+00:00,"-Edifício Joelma/SP, 1974.\n\n-Sgt CASSANIGA s...",None,None,video,0,jairbolsonaro,3154,16202,2020,7,27,20,51,0,0,0,1
2020-07-27 11:10:36+00:00,- Água para quem tem sede.\n- Liberdade para u...,None,None,video,0,jairbolsonaro,8101,37357,2020,7,27,11,10,0,0,0,1
2020-07-26 20:18:19+00:00,"@tarcisiogdf @MInfraestrutura 🤝🇧🇷, Ministro!",None,tarcisiogdf/MInfraestrutura,NaN,1,jairbolsonaro,1074,16840,2020,7,26,20,18,6,0,1,1
2020-07-26 15:40:39+00:00,2- @MinEconomia @MinCidadania @onyxlorenzoni @...,None,MinEconomia/MinCidadania/onyxlorenzoni/MEC_Com...,photo,1,jairbolsonaro,1337,6383,2020,7,26,15,40,6,0,1,1
2020-07-26 15:39:47+00:00,1- Acompanhe as redes sociais! @secomvc @fabio...,None,secomvc/fabiofaria5555/tarcisiogdf/MInfraestru...,photo,0,jairbolsonaro,3287,14836,2020,7,26,15,39,6,0,1,1


In [4]:
pd.set_option('max_colwidth',150)
initial_data = pd.DataFrame(tweeys_data['full_text'].values,index=range(0,len(tweeys_data)),columns=['text'])
initial_data.head()

,text
0,"-Edifício Joelma/SP, 1974.\n\n-Sgt CASSANIGA salta de helicóptero da FAB no terraço do edifício em chamas para salvar vidas, em uma das maiores tr..."
1,"- Água para quem tem sede.\n- Liberdade para um povo. \n- Brasil acima de tudo, Deus acima de todos!\n- BOM DIA.\n\n. YouTube: https://t.co/eS5aHQ..."
2,"@tarcisiogdf @MInfraestrutura 🤝🇧🇷, Ministro!"
3,2- @MinEconomia @MinCidadania @onyxlorenzoni @MEC_Comunicacao @ItamaratyGovBr @ernestofaraujo https://t.co/WVf6V7pqzC
4,1- Acompanhe as redes sociais! @secomvc @fabiofaria5555 @tarcisiogdf @MInfraestrutura @MinEconomia @minsaude @Mapa_Brasil @TerezaCrisMS @DHumanosB...


Limpando um pouco nosso texto de cada tweet:

In [5]:
def clean_text(text):
    text = text.lower() # texto minúsculo
    text = re.sub('[^A-zÀ-ú0-9 ]', '', text) # manter alfanuméricos e espaço
    text = re.sub('\w*\d\w*', '', text) # remover qualquer palavra que contenha numeros
    text = re.sub('\w*https?\w*', '', text) # remover urls
    text = ''.join(ch for ch in unicodedata.normalize('NFKD', text) 
        if not unicodedata.combining(ch))   # remover acentos
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remover pontuações esquisitas
    return text

clean_data = pd.DataFrame(initial_data.text.apply(lambda x: clean_text(x)))
clean_data.head()

,text
0,edificio joelmasp cassaniga salta de helicoptero da fab no terraco do edificio em chamas para salvar vidas em uma das maiores tragedias na histor...
1,agua para quem tem sede liberdade para um povo brasil acima de tudo deus acima de todos bom dia youtube
2,tarcisiogdf minfraestrutura ministro
3,mineconomia mincidadania onyxlorenzoni meccomunicacao itamaratygovbr ernestofaraujo
4,acompanhe as redes sociais secomvc tarcisiogdf minfraestrutura mineconomia minsaude mapabrasil terezacrisms dhumanosbrasil damaresalves


Criando a Document Term Matrix:

In [6]:
pt_stop_words = np.array([])
# stopwords.txt é o arquivo que se encontra no link no começo da subseção
with open("../../data/stopwords.txt",encoding="utf-8") as file:
    line = file.readline()
    while line != '':
        line = re.sub('\\n$| $', '', line)
        pt_stop_words = np.append(pt_stop_words,line)
        line = file.readline()
pt_stop_words = [clean_text(pt_stop_words[i]) for i in range(0,len(pt_stop_words))]
pt_stop_words = list(set(pt_stop_words))

cv = text.CountVectorizer(stop_words = text.ENGLISH_STOP_WORDS.union(pt_stop_words))
data_cv = cv.fit_transform(clean_data.values.ravel())
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = clean_data.index
tdm = data_dtm.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,12824,12825,12826,12827,12828,12829,12830,12831,12832,12833
aacd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aassibarreto,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abaete,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Aplicando Topic Modelling (tentativa 1)

Com a nossa Document Term Matrix (transposta devido à entrada da função a seguir), podemos usar a biblioteca Gensim para criar um modelo de Latent Dirichlet Allocation (LDA) e obter os nossos resultados.

In [7]:
# Colocando a Term Document Matrix em um novo formato gensim, de df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [8]:
# Gensim também precisa de um dicionário com todos os termos e suas respectivas posições na Term Document Matrix
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Agora que temos o corpus (Term Document Matrix) e o id2word (dicionário de localização: termo), precisamos especificar dois outros parâmetros - o número de tópicos e o número de passos. Inicialmente vamos testar com $4$, $6$ e $10$ tópicos juntamente com $10$ passos e ver se os resultados fazem sentido.

In [9]:
# Agora que já temos o corpus (Term Document Matrix) e id2word (dicionário de localização: termo),
# precisamos especificar também dois outros parâmetros - o número de tópicos e de passos
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10, random_state=42)
lda.print_topics()

[(0,
  '0.019*"rt" + 0.012*"jairbolsonaro" + 0.011*"presidente" + 0.010*"brasil" + 0.010*"bolsonaro" + 0.008*"todos" + 0.006*"dia" + 0.005*"sempre" + 0.005*"tudo" + 0.005*"deus"'),
 (1,
  '0.016*"rt" + 0.006*"sobre" + 0.006*"bolsonarosp" + 0.006*"rio" + 0.005*"jairbolsonaro" + 0.004*"bolsonaro" + 0.004*"presidente" + 0.004*"prudencia" + 0.004*"sofisticacao" + 0.004*"brasil"'),
 (2,
  '0.015*"governo" + 0.012*"brasil" + 0.008*"milhoes" + 0.008*"bolsonaro" + 0.006*"rt" + 0.006*"mil" + 0.005*"ano" + 0.004*"bilhoes" + 0.004*"federal" + 0.004*"pais"'),
 (3,
  '0.011*"rt" + 0.007*"bolsonaro" + 0.007*"esquerda" + 0.006*"ai" + 0.006*"pra" + 0.005*"pt" + 0.005*"contra" + 0.003*"pode" + 0.003*"fakenews" + 0.003*"sempre"')]

In [10]:
lda2 = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=6, passes=10, random_state=42)
lda2.print_topics()

[(0,
  '0.027*"rt" + 0.015*"jairbolsonaro" + 0.013*"presidente" + 0.009*"bolsonaro" + 0.007*"sempre" + 0.006*"abraco" + 0.006*"dia" + 0.006*"brasil" + 0.005*"verdade" + 0.005*"tudo"'),
 (1,
  '0.024*"rt" + 0.016*"bolsonarosp" + 0.010*"sobre" + 0.010*"bolsonaro" + 0.009*"presidente" + 0.006*"jairbolsonaro" + 0.006*"agora" + 0.006*"hoje" + 0.005*"live" + 0.004*"claro"'),
 (2,
  '0.011*"bolsonaro" + 0.009*"rt" + 0.008*"contra" + 0.008*"presidente" + 0.006*"fake" + 0.005*"news" + 0.005*"sobre" + 0.004*"pt" + 0.004*"brasil" + 0.004*"esquerda"'),
 (3,
  '0.012*"rt" + 0.010*"pra" + 0.005*"prudencia" + 0.005*"sofisticacao" + 0.005*"odio" + 0.005*"bolsonarosp" + 0.004*"politicos" + 0.004*"esquerda" + 0.004*"jornalista" + 0.004*"bolsonaro"'),
 (4,
  '0.019*"brasil" + 0.014*"governo" + 0.009*"bolsonaro" + 0.007*"pais" + 0.007*"ano" + 0.005*"todos" + 0.005*"rt" + 0.005*"maior" + 0.005*"ai" + 0.005*"jairbolsonaro"'),
 (5,
  '0.015*"milhoes" + 0.012*"governo" + 0.011*"mil" + 0.009*"programa" + 0.009

In [11]:
lda3 = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, passes=10, random_state=42)
lda3.print_topics()

[(0,
  '0.055*"rt" + 0.041*"jairbolsonaro" + 0.019*"presidente" + 0.012*"abraco" + 0.010*"carlosbolsonaro" + 0.009*"nunca" + 0.009*"bolsonaro" + 0.009*"hoje" + 0.008*"liberdade" + 0.008*"ver"'),
 (1,
  '0.018*"rio" + 0.018*"tarcisiogdf" + 0.017*"minfraestrutura" + 0.009*"bolsonaro" + 0.009*"sociais" + 0.008*"redes" + 0.007*"rt" + 0.007*"sobre" + 0.007*"obras" + 0.007*"janeiro"'),
 (2,
  '0.017*"bolsonaro" + 0.014*"presidente" + 0.014*"rt" + 0.010*"contra" + 0.007*"fake" + 0.007*"qualquer" + 0.007*"pode" + 0.007*"sobre" + 0.007*"governo" + 0.006*"jairbolsonaro"'),
 (3,
  '0.018*"rt" + 0.017*"pra" + 0.014*"arthurweint" + 0.012*"congresso" + 0.009*"bolsonaro" + 0.008*"pouco" + 0.008*"quanto" + 0.006*"projeto" + 0.006*"bolsonarosp" + 0.005*"mundo"'),
 (4,
  '0.025*"brasil" + 0.014*"todos" + 0.011*"dia" + 0.011*"deus" + 0.011*"pais" + 0.010*"presidente" + 0.009*"tudo" + 0.008*"sempre" + 0.008*"grande" + 0.007*"vai"'),
 (5,
  '0.016*"narrativa" + 0.013*"youtube" + 0.011*"link" + 0.010*"educa

Os resultados são interessantes, mas ainda confusos devido a algumas outras stop words e palavras. Realizaremos a mesma análise agora deixando apenas substantivos e adjetivos (e removendo algumas stop words extras).

## Aplicando Topic Modelling (tentativa 2)

Para que o nosso modelo se torne mais preciso para nossos objetivos, podemos adicionar algumas stop_words identificadas nos grupos de tópicos acima e também filtrar nossas palavras para apenas substantivos e adjetivos.

In [12]:
# Algumas stop words comuns nos tweets
add_stop_words = ['pra','ai','rt', 'fazem', 'aqui', 'entao', 'vamos', 'vai', 'onde']

# Função que retorna substantivo e adjetivo
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = nltk.word_tokenize(text,language='portuguese')
    nouns_adj = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

nouns_adj_data = pd.DataFrame(clean_data.text.apply(lambda x: nouns_adj(x)))
nouns_adj_data.head()

,text
0,edificio joelmasp cassaniga salta da fab terraco chamas para salvar uma das maiores historia youtube
1,agua para quem tem sede para um povo brasil acima deus acima bom dia youtube
2,tarcisiogdf minfraestrutura ministro
3,mineconomia mincidadania onyxlorenzoni meccomunicacao itamaratygovbr ernestofaraujo
4,acompanhe redes secomvc tarcisiogdf minfraestrutura mineconomia mapabrasil terezacrisms damaresalves


In [13]:
cv_new = text.CountVectorizer(stop_words = text.ENGLISH_STOP_WORDS.union(pt_stop_words+add_stop_words))
data_cv_new = cv_new.fit_transform(nouns_adj_data.values.ravel())
data_dtm_new = pd.DataFrame(data_cv_new.toarray(), columns=cv_new.get_feature_names())
data_dtm_new.index = nouns_adj_data.index
tdm_new = data_dtm_new.transpose()
sparse_counts_new = scipy.sparse.csr_matrix(tdm_new)

# Criando o corpus do Gensim
corpus_new = matutils.Sparse2Corpus(sparse_counts_new)

# Create the vocabulary dictionary
id2word_new = dict((v, k) for k, v in cv_new.vocabulary_.items())

In [14]:
# Teste final com 10 tópicos
lda6 = models.LdaModel(corpus=corpus_new, num_topics=10, id2word=id2word_new, passes=20, random_state=42)
lda6.print_topics()

[(0,
  '0.016*"news" + 0.012*"brasil" + 0.012*"fake" + 0.008*"sobre" + 0.008*"folha" + 0.006*"eua" + 0.006*"jairbolsonaro" + 0.006*"entrevista" + 0.006*"renovamidia" + 0.006*"tv"'),
 (1,
  '0.012*"dia" + 0.011*"verdade" + 0.011*"presidente" + 0.011*"abraco" + 0.011*"bolsonaro" + 0.009*"todos" + 0.009*"bom" + 0.008*"grande" + 0.007*"narrativa" + 0.007*"brasil"'),
 (2,
  '0.029*"brasil" + 0.011*"todos" + 0.010*"deus" + 0.008*"parabens" + 0.007*"pais" + 0.007*"jairbolsonaro" + 0.007*"esquerda" + 0.007*"arthurweint" + 0.006*"bem" + 0.005*"dar"'),
 (3,
  '0.012*"bolsonaro" + 0.010*"jairbolsonaro" + 0.009*"boa" + 0.008*"brasil" + 0.008*"rio" + 0.007*"presidente" + 0.007*"pode" + 0.006*"falar" + 0.006*"querem" + 0.006*"pt"'),
 (4,
  '0.025*"jairbolsonaro" + 0.019*"minfraestrutura" + 0.016*"tarcisiogdf" + 0.011*"youtube" + 0.010*"alguem" + 0.009*"dias" + 0.008*"exercitooficial" + 0.007*"govbr" + 0.007*"link" + 0.006*"brazil"'),
 (5,
  '0.014*"brasil" + 0.010*"jairbolsonaro" + 0.009*"mineconomi

Como podemos ver, os assuntos principais dos tweets de cada grupo de tópicos se envolvem em torno de:

0. Fake news e mídia internacional;
1. Saudações diárias; 
2. Parabenizações;
3. Críticas à esquerda;
4. Infraestrutura e exército;
5. Economia e comunicação;
6. Opiniões sobre notícias;
7. Direcionados ao governador de SP João Doria;
8. Oposição da mídia;
9. Relacionados com municípios menores;

Claro que todas essas afirmações acima são completamente qualitativas.

Colocando no nosso dataframe a classificação de tópicos:

In [15]:
corpus_transformed = lda6[corpus_new]
topics = []
for tweetTopics in corpus_transformed:
    maxValue = 0
    rightTopic = 0
    for topic,value in tweetTopics:
        if(value>maxValue):
            maxValue = value
            rightTopic = topic
    topics.append(rightTopic)
tweeys_data['topic'] = np.array(topics)

Exportando nossas variáveis para próximos usos:

In [16]:
topicsWords = lda6.show_topics()
topicsDf = []
for topicoId, words in topicsWords:
    for word in words.split(sep='+'):
        topicsDf.append([topicoId,word[0:5],word[7:].strip(' ').strip('"')])
topicsDf = pd.DataFrame(topicsDf,columns=['topic','weight','word'])
topicsDf.head()

,topic,weight,word
0,0,0.016,news
1,0,0.01,brasil
2,0,0.01,fake
3,0,0.00,sobre
4,0,0.00,folha
